In [1]:
import math
import random
import numpy as np
import os
import fnmatch
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import shapelets as sha
import importlib
importlib.reload(sha)
from operator import itemgetter
random.seed(1)
lcs=[]
classes=[]
xs=np.arange(0,30)
def noisy(average_value):
    return average_value+(math.cos(random.randint(0,360)*(math.pi/180))*0.1*average_value)
no_per_class=50
for i in range(no_per_class):
    ys=[]
    peak=np.random.choice(xs[1:-1])
    for x in xs:
        y=noisy(2)
        ys.append(y)
    ys[peak]=ys[peak-1]=ys[peak+1]=noisy(5)
    lcs.append(np.stack((xs,ys)))
    classes.append("alpha")
for i in range(no_per_class):
    ys=[]
    peak=np.random.choice(xs[:-1])
    for x in xs:
        y=noisy(2)
        ys.append(y)
    ys[peak]=noisy(10)
    lcs.append(np.stack((xs,ys)))
    classes.append("beta")
for i in range(no_per_class):
    ys=[]
    peak=np.random.choice(xs[:-3])
    for x in xs:
        y=noisy(2)
        ys.append(y)
    ys[peak]=noisy(10)
    #if peak<xs[-2]:
    ys[peak+2]=ys[peak+3]=noisy(5)
    lcs.append(np.stack((xs,ys)))
    classes.append("gamma")
for i in range(no_per_class):
    ys=[]
    #peak=np.random.choice(xs[:-3])
    for x in xs:
        y=noisy(2)
        ys.append(y)
    #ys[peak]=noisy(10)
    #if peak<xs[-2]:
    #ys[peak+2]=ys[peak+3]=noisy(5)
    lcs.append(np.stack((xs,ys)))
    classes.append("delta")
ids=[]
for i in range(len(classes)):
    ids.append(i)
ob_state = {}
for i, ob in enumerate(classes):
    ob_state[i] = ob

In [2]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test, id_train, id_test = train_test_split(lcs, classes, ids, test_size=0.5, random_state=1, stratify=classes)

In [3]:
importlib.reload(sha)
best_shapelets=[]
time_res=1
x=[]
for n_donor, lc_donor in enumerate(x_train):
    #Create lists with classifications of all time-series relative to the donor time series; one that the pool of shapelets is generated from
    state_donor = y_train[n_donor]#y_train[n],x_train[n] and id_train[n] all refer to the attributes of the same time series
    belong_class=[]
    other_class=[]
    for n, i in enumerate(id_train):
        if y_train[n] == state_donor:
            belong_class.append(i)
        else:
            other_class.append(i)
    #calculate the entropy of the entire set, so it can be compared to the split set later
    prop_belong = len(belong_class)/(len(belong_class)+len(other_class))
    set_entropy = -(prop_belong)*math.log2(prop_belong)-(1-prop_belong)*math.log2(1-prop_belong)
    pool=sha.generate_shapelets(lc_donor, 1, len(lc_donor[0]))#generate shapelets from the donor time-series, 
    #set the initial best value of information gain to 0 (improved by any split) and start testing the shapelets
    best_gain=0
    for shapelet in pool:
        skip_shapelet=False#for entropy pruning
        #set the order of distance calculations
        #pick an other_class object first and then alternate between belong and other, when one group runs out, append the rest of the other group to the end
        order=[]
        if len(belong_class)<len(other_class):alternations=len(belong_class);larger_group=other_class
        else: alternations=len(other_class); larger_group=belong_class
        for i in range(alternations):
            order.append(other_class[i])
            order.append(belong_class[i])
        for i in range(len(larger_group)-alternations):
            order.append(larger_group[-(i+1)])
        #start distance calculations
        distances=[]
        for n_lc in order:
            if id_train[n_donor] == n_lc:
                distance = 0
            else:
                lc=x_train[np.where(np.array(id_train)==n_lc)[0][0]]
                distance=sha.distance_calculation(shapelet, lc, early_abandon=True)
            #save the distance value together with the classification and lightcurve id
            if n_lc in belong_class:
                class_assign=1
            else:
                class_assign=0
            distances.append((n_lc ,distance, class_assign))
            #find the optimal split point if there are at least two distances calculated, then use entropy pruning to find if the shapelet still has a change to beat the best one found so far
            if len(distances)>1:
                best_split=sha.best_split_point(distances, set_entropy)
                skip_shapelet=sha.entropy_pruning(best_gain, distances, best_split, len(belong_class), len(other_class), set_entropy)
                #x.append((best_split, distances))

KeyboardInterrupt: 

In [150]:
from copy import deepcopy
distances=[[-1,1,0],[-1,10,1],[-1,15,1],[-1,8,1],
          [-1,11,0],[-1,10,0],[-1,3,1],[-1,7,1]]
prop_belong=(above_belong+below_belong)/len(distances)
split_point=10
set_entropy = -(prop_belong)*math.log2(prop_belong)-(1-prop_belong)*math.log2(1-prop_belong)
#split_point=sha.best_split_point(distances, set_entropy)
above=[lc for lc in distances if lc[1]>=split_point]
above_belong=sum([lc[2] for lc in above])
below=[lc for lc in distances if lc[1]<split_point]
below_belong=sum([lc[2] for lc in below])
prop_above_belong=above_belong/len(above)
prop_below_belong=below_belong/len(below)
if prop_above_belong==1. or prop_above_belong==0.:
    above_entropy=0
else:
    above_entropy = -(prop_above_belong)*math.log2(prop_above_belong)-(1-prop_above_belong)*math.log2(1-prop_above_belong)
if prop_below_belong==1. or prop_below_belong==0.:
    below_entropy =0
else:
    below_entropy = -(prop_below_belong)*math.log2(prop_below_belong)-(1-prop_below_belong)*math.log2(1-prop_below_belong)
info_gain=set_entropy-(len(above)/(len(above)+len(below)))*(above_entropy)-(len(below)/(len(above)+len(below)))*(below_entropy)
print(above_belong, below_belong, prop_above_belong, prop_below_belong)
print(above_entropy,below_entropy)
print(info_gain)
best_gain=0.09235938389499487
best_split=split_point
belong_class_count=sum([lc[2] for lc in distances])
other_class_count=len(distances)-belong_class_count


calc_belong=sum([lc[2] for lc in distances])
calc_other=len(distances)-calc_belong
distances_bcs=deepcopy(distances) #best case scenario when all the distances are included
distances_bcs.sort(key=itemgetter(1))
maxdist=distances_bcs[-1][1]+1
for add_belong in range(belong_class_count-calc_belong):
    distances_bcs.append((-1,0,1))
for add_other in range(other_class_count-calc_other):
    distances_bcs.append((-1,maxdist,0))
gain_bcs=sha.information_gain(distances_bcs, set_entropy, best_split)
if isinstance(gain_bcs, str) == True:
    ans= True
else:
    if gain_bcs<best_gain:
        ans= True
    else:
        ans= False
ans

2 3 0.5 0.75
1.0 0.8112781244591328
0.048794940695398525


True

In [142]:
print(above_belong, below_belong, prop_above_belong, prop_below_belong)
print(above_entropy,below_entropy)
print(info_gain)

1 4 0.5 0.6666666666666666
1.0 0.9182958340544896
0.015712127384097774


In [106]:
set_entropy

0.8112781244591328

{'a': 5, 'b': 5}


In [138]:
prop_above_belong=0.33333
-(prop_above_belong)*math.log2(prop_above_belong)-(1-prop_above_belong)*math.log2(1-prop_above_belong)

0.9182925006850887

In [166]:
distances=[[-5,1,0],[-5,1,0],[-5,2,0],[-5,1,0],[-5,1,0],
          [-5,11,1],[-5,10,1],[-5,15,1],[-5,7,1],[-5,7,1]]
best_gain=0.04556599707503506
best_split=4.5
belong_class_count=10
other_class_count=10
set_entropy=1

def entropy_pruning(best_gain, distances, best_split, belong_class_count, other_class_count, set_entropy):
    calc_belong=sum([lc[2] for lc in distances])
    calc_other=len(distances)-calc_belong
    distances_bcs=deepcopy(distances) #best case scenario when all the distances are included
    distances_bcs.sort(key=itemgetter(1))
    maxdist=distances_bcs[-1][1]+1
    print(calc_belong, calc_other, maxdist)
    for add_belong in range(belong_class_count-calc_belong):
        distances_bcs.append((-1,0,1))
    for add_other in range(other_class_count-calc_other):
        distances_bcs.append((-1,maxdist,0))
    print(distances_bcs)
    gain_bcs=sha.information_gain(distances_bcs, set_entropy, best_split)
    print(gain_bcs)
    if isinstance(gain_bcs, str) == True:
        return True
    else:
        if gain_bcs<best_gain:
            return True
        else:
            return False

entropy_pruning(best_gain, distances, best_split, belong_class_count, other_class_count, set_entropy)        

5 5 16
[[-5, 1, 0], [-5, 1, 0], [-5, 1, 0], [-5, 1, 0], [-5, 2, 0], [-5, 7, 1], [-5, 7, 1], [-5, 10, 1], [-5, 11, 1], [-5, 15, 1], (-1, 0, 1), (-1, 0, 1), (-1, 0, 1), (-1, 0, 1), (-1, 0, 1), (-1, 16, 0), (-1, 16, 0), (-1, 16, 0), (-1, 16, 0), (-1, 16, 0)]
0.0


True